In [12]:
import mne
import os
import sys
sys.path.insert(0,'../')
from libs.ssl_dataloader import *
from libs.ssl_model import *
from libs.ssl_utils import *
from libs import eeg_utils
from sklearn.model_selection import train_test_split
# Use these commands in the same cell to autoreload local library source code
# https://stackoverflow.com/questions/45807760/change-source-code-and-update-jupyter-notebook-but-retain-variables
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def get_subjects(basedir):
    files = np.array([i for i in os.listdir(basedir) if i.split('.')[-1] == 'set'])
    subjects = np.array([i.split('_')[0] for i in os.listdir(basedir) if i.split('.')[-1] == 'set'])
    return subjects, files

subjects, files = get_subjects("/mnt/nemar/child-mind-rest")
subj_train, subj_test, files_train, files_test = train_test_split(subjects, files, test_size=0.1, random_state=1)

subj_train, subj_val, files_train, files_val = train_test_split(subj_train, files_train, test_size=0.22, random_state=1) # 0.22 x 0.9 = 0.2

In [3]:
SFREQ = 128
pos_nsec = neg_nsec = 6
transformer = MaskedContrastiveLearning.SegmentInput(M=SFREQ/4)
dataset = ChildmindSSLDataset(
        data_dir = "/mnt/nemar/child-mind-rest",
        # subjects = subj_train.tolist(),
        n_subjects=10,
        x_params = {
            "feature": "RelativePositioning",
            "cache_dir": '/mnt/nemar/dtyoung/eeg-ssl/data/childmind-rest-cache',
            "win": SFREQ*2,
            "stride": SFREQ,
            "tau_pos": int(SFREQ*pos_nsec),
            "tau_neg": int(SFREQ*neg_nsec),
            "isResume": True
        },
        x_transformer=transformer
    )

100%|██████████| 10/10 [00:00<00:00, 34267.19it/s]


/home/dung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:70: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  EEG = mne.io.read_raw_eeglab(os.path.join(raw_file), preload=True)
/home/dung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:70: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  EEG = mne.io.read_raw_eeglab(os.path.join(raw_file), preload=True)
/home/dung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:70: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  EEG = mne.io.read_raw_eeglab(os.path.join(raw_file), preload=True)
/home/dung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:70: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epochi

In [18]:
print('Length of dataset', len(dataset))
print('X dim', dataset[0][0].shape)
print('Y dim', dataset[0][1].shape)
# eeg_utils.plot_raw_eeg(dataset[2][0][1])

Length of dataset 1680
X dim (2, 128, 256)
Y dim ()


In [11]:
model_utils = SSLModelUtils(model_params={'model': 'VGGSSL', 'task': 'RP'}, train_params={'num_epochs': 15})
model_utils.train(dataset)

Epoch 0, Iteration 0, loss = 0.6911
Epoch 0, Iteration 10, loss = 0.6949
Epoch 0, Iteration 20, loss = 0.6892
Epoch 0, Iteration 30, loss = 0.6926
Epoch 0, Iteration 40, loss = 0.6884
Epoch 0, Iteration 50, loss = 0.6806
Epoch 0, Iteration 60, loss = 0.7033
Epoch 0, Iteration 70, loss = 0.6911
Epoch 0, Iteration 80, loss = 0.6926
Epoch 0, Iteration 90, loss = 0.6549
Epoch 0, Iteration 100, loss = 0.6907
Epoch 1, Iteration 0, loss = 0.7044
Epoch 1, Iteration 10, loss = 0.6929
Epoch 1, Iteration 20, loss = 0.6775
Epoch 1, Iteration 30, loss = 0.6852
Epoch 1, Iteration 40, loss = 0.6720
Epoch 1, Iteration 50, loss = 0.6717
Epoch 1, Iteration 60, loss = 0.6885
Epoch 1, Iteration 70, loss = 0.7098
Epoch 1, Iteration 80, loss = 0.6913
Epoch 1, Iteration 90, loss = 0.6906
Epoch 1, Iteration 100, loss = 0.7040
Epoch 2, Iteration 0, loss = 0.6991
Epoch 2, Iteration 10, loss = 0.6851
Epoch 2, Iteration 20, loss = 0.6941
Epoch 2, Iteration 30, loss = 0.6288
Epoch 2, Iteration 40, loss = 0.6979
Ep